# SAT with z3: tutorial and examples

## Getting Started with Z3

[Z3](https://github.com/Z3Prover/z3) is a state-of-the-art theorem prover developed at Microsoft Research and used in many real-world applications such as: software/hardware verification and testing, constraint solving, analysis of hybrid systems, security, and geometrical problems.

The Z3 input format is an extension of the [SMT-LIB 2.0 standard](http://www.smtlib.org/), but APIs for common programming languages like `Python`, `Java`, `C`, and `.Net` are officially available in order to ease implementation and favor its use as a component in the context of other tools that require solving logical formulas. Unofficial porting for functional languages like [`Scala`](https://github.com/epfl-lara/ScalaZ3) or [`Haskell`](https://github.com/LeventErkok/sbv) are also available.

For the sake of simplicity, in this tutorial we will explore how to use Z3Py (the Z3 API in Python), which is the programming language the reader is probably most comfortable with.

At the time of writing, the latest version of Z3 is the `4.8.X`, which is the one used here. Any compatibility issues you may find running this notebook can be related to a different version of the tool.

## Installation

The easiest way to install Z3Py, along with the Z3 binary, is to use Python's package manager `pip`.

```bash
pip install z3-solver
```

In order to check if Z3 has been installed properly, open a terminal and type:

```bash
z3 --version
```

If the output is something like _Z3 version 4.8.X - 64 bit_, you are good to go.

## Startup
In order to use z3 in your projects or in your scripts, you need to import it first.

In [1]:
from z3 import *

## Boolean Logic

As stated before, to shape a SAT problem we need the boolean operators. 

Z3, being a SAT solver supports all the boolean operators: `And`, `Or`, `Not`, `Implies`, `If`, `==` (bi-implication). 

The following example shows how to create a boolean formula.

In [2]:
p = Bool('P')
q = Bool('Q')
r = Bool('R')

formula = And(Implies(p, q), r == Not(q), Or(Not(p), r))
formula

And(Implies(P, Q), R == Not(Q), Or(Not(P), R))

You can use the `solve` method to find an assignment of the variables which satisfies the formula.

In [3]:
solve(formula)

[R = False, Q = True, P = False]


`And` operator is implicitly considered by the `solve` function because all the clauses must be valid at the same time, therefore we can omit it.

In [4]:
solve(Implies(p, q), r == Not(q), Or(Not(p), r))

[R = False, Q = True, P = False]


## Solvers

The command `solve`, used in the previous examples, is implemented using the Z3 solver API.

The command `Solver` creates a general purpose solver. 

In [6]:
p = Bool('P')
q = Bool('Q')
r = Bool('R')

s = Solver()

Constraints can be added using the method `add`. We say the constraints have been asserted in the solver.

In [7]:
s.add(Or(p, q))
s

[Or(P, Q)]

The method `check` solves the asserted constraints. The result is `sat` (satisfiable) if a solution was found, `unsat` (unsatisfiable) if no solution exists, or `unknown` if the solver fails to return for some other reason...

In [8]:
print("Solving constraints in the solver s...")
print(s.check())
s.model()

Solving constraints in the solver s...
sat


[Q = False, P = True]

In some applications, we want to explore several similar problems that share several constraints. We can use the commands `push` and `pop` for doing that. 

Each solver maintains a stack of assertions. The command `push` creates a new scope by saving the current stack size. The command `pop` removes any assertion performed between it and the matching push. The `check` method always operates on the content of solver assertion stack.

In [9]:
print("Create a new scope...")
s.push()
s.add(And(q, p))
s

Create a new scope...


[Or(P, Q), And(Q, P)]

In [10]:
print("Solving updated set of constraints...")
print(s.check())
s.model()

Solving updated set of constraints...
sat


[Q = True, P = True]

In [11]:
print("Restoring state...")
s.pop()
print(s.check())
s

Restoring state...
sat


[Or(P, Q)]

### Proofs

Until now, we have talked about satisfiability, determining an assignment of appropriate values under which our series of constraints evaluates to true.

Instead, we can be interested in finding a proof of a statement $F$, i.e., whether it is valid for any combination of values. In order to do that, we can exploit the fact that if $F$ is always true, $\mbox{Not}(F)$ is always false, therefore trying to satisfy it gives `unsat` as result.

The following example use this trick to prove De Morgan's law.

In [12]:
p, q = Bools('p q')
demorgan = And(p, q) == Not(Or(Not(p), Not(q)))
demorgan

And(p, q) == Not(Or(Not(p), Not(q)))

In [13]:
print("Proving De Morgan...")
prove(demorgan)

Proving De Morgan...
proved


## List comprehensions

Python supports list comprehensions. List comprehensions provide a concise way to create lists. They can be used to create Z3 expressions and problems in Z3Py. 

The following example demonstrates how to use Python list comprehensions in Z3Py.

In [14]:
# Create a list containing 5 integer variables
X = [Bool(f'x{i}') for i in range(5)]
print(X)

[x0, x1, x2, x3, x4]


## Additional Resources

* [Official repo](https://github.com/z3prover/z3)
* [SMT-LIB 2.0 tutorial](https://rise4fun.com/z3/tutorial)
* [More examples using Z3Py](https://ericpony.github.io/z3py-tutorial/)
* [Mastering Z3](https://theory.stanford.edu/~nikolaj/programmingz3.html)